# Risk Analysis: VaR and Volatility Calculations

This notebook demonstrates various risk metrics including Value at Risk (VaR), Conditional VaR (CVaR), volatility analysis, and other risk measures.

## Topics Covered

1. Historical volatility calculation
2. Value at Risk (VaR) - Historical, Parametric, and Monte Carlo methods
3. Conditional VaR (Expected Shortfall)
4. Beta calculation
5. Maximum drawdown analysis
6. Risk visualization techniques

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')

## 1. Fetch Data for Risk Analysis

In [ ]:
# Define assets for analysis
ticker = 'AAPL'  # Primary asset
market_ticker = 'SPY'  # Market proxy

# Download 3 years of data
end_date = datetime.now()
start_date = end_date - timedelta(days=3*365)

print(f"Downloading data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

# Fetch data
data = yf.download([ticker, market_ticker], start=start_date, end=end_date)['Adj Close']

# Calculate returns
returns = data.pct_change().dropna()
asset_returns = returns[ticker]
market_returns = returns[market_ticker]

print(f"\nData shape: {data.shape}")
print(f"Returns shape: {returns.shape}")
returns.head()

## 2. Basic Risk Statistics

In [ ]:
# Calculate basic statistics
stats_dict = {
    'Mean Daily Return': asset_returns.mean(),
    'Daily Volatility': asset_returns.std(),
    'Annual Return': asset_returns.mean() * 252,
    'Annual Volatility': asset_returns.std() * np.sqrt(252),
    'Skewness': asset_returns.skew(),
    'Kurtosis': asset_returns.kurtosis(),
    'Sharpe Ratio': (asset_returns.mean() * 252) / (asset_returns.std() * np.sqrt(252))
}

stats_df = pd.DataFrame.from_dict(stats_dict, orient='index', columns=['Value'])
print(f"Risk Statistics for {ticker}:")
print(stats_df.round(4))

## 3. Volatility Analysis

In [ ]:
# Calculate rolling volatility
rolling_windows = [20, 60, 252]  # 1-month, 3-month, 1-year

fig, ax = plt.subplots(figsize=(14, 8))

for window in rolling_windows:
    rolling_vol = asset_returns.rolling(window=window).std() * np.sqrt(252)
    ax.plot(rolling_vol, label=f'{window}-day ({window//20:.0f} month)', linewidth=2)

ax.set_title(f'Rolling Volatility Analysis - {ticker}', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Annualized Volatility', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

# Format y-axis as percentage
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

plt.tight_layout()
plt.show()

## 4. Value at Risk (VaR) Calculations

We'll calculate VaR using three methods:
1. Historical Method
2. Parametric (Variance-Covariance) Method
3. Monte Carlo Simulation

In [ ]:
# VaR parameters
confidence_levels = [0.95, 0.99]
holding_period = 1  # 1 day
initial_investment = 1000000  # $1 million

var_results = pd.DataFrame()

for confidence in confidence_levels:
    # 1. Historical VaR
    var_historical = np.percentile(asset_returns, (1 - confidence) * 100)
    
    # 2. Parametric VaR (assuming normal distribution)
    z_score = stats.norm.ppf(1 - confidence)
    var_parametric = asset_returns.mean() + z_score * asset_returns.std()
    
    # 3. Monte Carlo VaR
    np.random.seed(42)
    simulated_returns = np.random.normal(asset_returns.mean(), 
                                       asset_returns.std(), 
                                       10000)
    var_monte_carlo = np.percentile(simulated_returns, (1 - confidence) * 100)
    
    # Store results
    var_results[f'{confidence:.0%} Confidence'] = [
        var_historical * initial_investment,
        var_parametric * initial_investment,
        var_monte_carlo * initial_investment
    ]

var_results.index = ['Historical VaR', 'Parametric VaR', 'Monte Carlo VaR']
print(f"Value at Risk (1-day holding period, ${initial_investment:,} investment):")
print(var_results.round(2))

## 5. Conditional VaR (Expected Shortfall)

In [ ]:
# Calculate Conditional VaR (CVaR)
cvar_results = pd.DataFrame()

for confidence in confidence_levels:
    # Historical CVaR
    var_threshold = np.percentile(asset_returns, (1 - confidence) * 100)
    cvar_historical = asset_returns[asset_returns <= var_threshold].mean()
    
    # Parametric CVaR (assuming normal distribution)
    z_score = stats.norm.ppf(1 - confidence)
    cvar_parametric = asset_returns.mean() - asset_returns.std() * stats.norm.pdf(z_score) / (1 - confidence)
    
    cvar_results[f'{confidence:.0%} Confidence'] = [
        cvar_historical * initial_investment,
        cvar_parametric * initial_investment
    ]

cvar_results.index = ['Historical CVaR', 'Parametric CVaR']
print(f"Conditional Value at Risk (Expected Shortfall):")
print(cvar_results.round(2))

## 6. VaR Visualization

In [ ]:
# Visualize return distribution with VaR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogram with VaR lines
ax1.hist(asset_returns, bins=100, alpha=0.7, density=True, label='Historical Returns')

# Add normal distribution overlay
x = np.linspace(asset_returns.min(), asset_returns.max(), 100)
ax1.plot(x, stats.norm.pdf(x, asset_returns.mean(), asset_returns.std()), 
         'r-', linewidth=2, label='Normal Distribution')

# Add VaR lines
colors = ['orange', 'red']
for i, confidence in enumerate(confidence_levels):
    var_value = np.percentile(asset_returns, (1 - confidence) * 100)
    ax1.axvline(var_value, color=colors[i], linestyle='--', linewidth=2, 
               label=f'VaR {confidence:.0%} ({var_value:.2%})')

ax1.set_xlabel('Daily Returns', fontsize=12)
ax1.set_ylabel('Probability Density', fontsize=12)
ax1.set_title(f'Return Distribution with VaR - {ticker}', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Q-Q plot to check normality
stats.probplot(asset_returns, dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot (Normality Test)', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Beta Calculation

In [ ]:
# Calculate beta relative to market
covariance = np.cov(asset_returns, market_returns)[0, 1]
market_variance = np.var(market_returns)
beta = covariance / market_variance

# Alternative method using regression
slope, intercept, r_value, p_value, std_err = stats.linregress(market_returns, asset_returns)

# Visualization
plt.figure(figsize=(10, 8))
plt.scatter(market_returns, asset_returns, alpha=0.5)
plt.plot(market_returns, slope * market_returns + intercept, 'r-', linewidth=2, 
         label=f'Beta = {beta:.2f}')

plt.xlabel(f'{market_ticker} Returns', fontsize=12)
plt.ylabel(f'{ticker} Returns', fontsize=12)
plt.title(f'Security Characteristic Line: {ticker} vs {market_ticker}', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

# Add statistics text
textstr = f'R² = {r_value**2:.3f}\nAlpha = {intercept:.4f}\nBeta = {beta:.3f}'
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=12,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"Beta Analysis:")
print(f"Beta: {beta:.3f}")
print(f"Alpha (annualized): {intercept * 252:.2%}")
print(f"R-squared: {r_value**2:.3f}")

## 8. Maximum Drawdown Analysis

In [ ]:
# Calculate cumulative returns
cumulative_returns = (1 + asset_returns).cumprod()

# Calculate running maximum
running_max = cumulative_returns.expanding().max()

# Calculate drawdown
drawdown = (cumulative_returns - running_max) / running_max

# Find maximum drawdown
max_drawdown = drawdown.min()
max_drawdown_date = drawdown.idxmin()

# Find recovery date
recovery_date = None
if max_drawdown_date:
    post_drawdown = cumulative_returns[max_drawdown_date:]
    peak_value = running_max[max_drawdown_date]
    recovery_mask = post_drawdown >= peak_value
    if recovery_mask.any():
        recovery_date = post_drawdown[recovery_mask].index[0]

# Visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Plot cumulative returns
ax1.plot(cumulative_returns.index, cumulative_returns, label='Cumulative Returns', linewidth=2)
ax1.plot(running_max.index, running_max, label='Running Maximum', linestyle='--', alpha=0.7)
ax1.fill_between(cumulative_returns.index, cumulative_returns, running_max, 
                where=(cumulative_returns < running_max), alpha=0.3, color='red')
ax1.set_ylabel('Cumulative Returns', fontsize=12)
ax1.set_title(f'Cumulative Returns and Drawdowns - {ticker}', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot drawdown
ax2.fill_between(drawdown.index, 0, drawdown, color='red', alpha=0.3)
ax2.plot(drawdown.index, drawdown, color='red', linewidth=2)
ax2.axhline(y=max_drawdown, color='darkred', linestyle='--', 
           label=f'Max Drawdown: {max_drawdown:.2%}')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Drawdown', fontsize=12)
ax2.set_title('Drawdown Analysis', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Format y-axis as percentage
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

plt.tight_layout()
plt.show()

print(f"Maximum Drawdown Analysis:")
print(f"Maximum Drawdown: {max_drawdown:.2%}")
print(f"Date of Maximum Drawdown: {max_drawdown_date.strftime('%Y-%m-%d')}")
if recovery_date:
    recovery_days = (recovery_date - max_drawdown_date).days
    print(f"Recovery Date: {recovery_date.strftime('%Y-%m-%d')}")
    print(f"Recovery Period: {recovery_days} days")
else:
    print("Not yet recovered from maximum drawdown")

## 9. Risk-Adjusted Performance Metrics

In [ ]:
# Calculate various risk-adjusted metrics
risk_free_rate = 0.02  # 2% annual risk-free rate
daily_rf = risk_free_rate / 252

# Sharpe Ratio
excess_returns = asset_returns - daily_rf
sharpe_ratio = np.sqrt(252) * excess_returns.mean() / asset_returns.std()

# Sortino Ratio (using downside deviation)
downside_returns = asset_returns[asset_returns < 0]
downside_deviation = np.sqrt(252) * downside_returns.std()
sortino_ratio = np.sqrt(252) * excess_returns.mean() / downside_returns.std()

# Calmar Ratio
annual_return = asset_returns.mean() * 252
calmar_ratio = annual_return / abs(max_drawdown)

# Information Ratio (vs market)
active_returns = asset_returns - market_returns
tracking_error = active_returns.std() * np.sqrt(252)
information_ratio = (active_returns.mean() * 252) / tracking_error

# Create summary table
risk_metrics = pd.DataFrame({
    'Metric': ['Annual Return', 'Annual Volatility', 'Sharpe Ratio', 'Sortino Ratio', 
               'Calmar Ratio', 'Information Ratio', 'Beta', 'Maximum Drawdown'],
    'Value': [annual_return, asset_returns.std() * np.sqrt(252), sharpe_ratio, 
              sortino_ratio, calmar_ratio, information_ratio, beta, max_drawdown]
})

# Format the display
def format_metric(row):
    if row['Metric'] in ['Annual Return', 'Annual Volatility', 'Maximum Drawdown']:
        return f"{row['Value']:.2%}"
    else:
        return f"{row['Value']:.3f}"

risk_metrics['Formatted Value'] = risk_metrics.apply(format_metric, axis=1)
print("Risk-Adjusted Performance Metrics:")
print(risk_metrics[['Metric', 'Formatted Value']].to_string(index=False))

## 10. Risk Dashboard

In [ ]:
# Create a comprehensive risk dashboard
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Return Distribution
ax1 = fig.add_subplot(gs[0, 0])
ax1.hist(asset_returns, bins=50, alpha=0.7, color='blue', edgecolor='black')
ax1.axvline(asset_returns.mean(), color='red', linestyle='--', label='Mean')
ax1.axvline(np.percentile(asset_returns, 5), color='orange', linestyle='--', label='VaR 95%')
ax1.set_title('Return Distribution', fontsize=12)
ax1.set_xlabel('Daily Returns')
ax1.legend(fontsize=10)

# 2. Rolling Volatility
ax2 = fig.add_subplot(gs[0, 1])
rolling_vol = asset_returns.rolling(window=30).std() * np.sqrt(252)
ax2.plot(rolling_vol, color='green', linewidth=2)
ax2.set_title('30-Day Rolling Volatility', fontsize=12)
ax2.set_ylabel('Annualized Volatility')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

# 3. VaR Over Time
ax3 = fig.add_subplot(gs[0, 2])
rolling_var = asset_returns.rolling(window=252).apply(lambda x: np.percentile(x, 5))
ax3.plot(rolling_var, color='red', linewidth=2)
ax3.set_title('Rolling 1-Year VaR (95%)', fontsize=12)
ax3.set_ylabel('VaR')
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))

# 4. Cumulative Returns vs Market
ax4 = fig.add_subplot(gs[1, :])
cum_asset = (1 + asset_returns).cumprod()
cum_market = (1 + market_returns).cumprod()
ax4.plot(cum_asset, label=ticker, linewidth=2)
ax4.plot(cum_market, label=market_ticker, linewidth=2)
ax4.set_title('Cumulative Returns Comparison', fontsize=12)
ax4.set_ylabel('Cumulative Returns')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Drawdown
ax5 = fig.add_subplot(gs[2, 0:2])
ax5.fill_between(drawdown.index, 0, drawdown, color='red', alpha=0.3)
ax5.plot(drawdown, color='red', linewidth=1)
ax5.set_title('Drawdown Analysis', fontsize=12)
ax5.set_ylabel('Drawdown')
ax5.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

# 6. Risk Metrics Table
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')
table_data = [
    ['Sharpe Ratio', f'{sharpe_ratio:.3f}'],
    ['Sortino Ratio', f'{sortino_ratio:.3f}'],
    ['Beta', f'{beta:.3f}'],
    ['Max Drawdown', f'{max_drawdown:.2%}'],
    ['VaR (95%)', f'{np.percentile(asset_returns, 5):.2%}'],
    ['CVaR (95%)', f'{asset_returns[asset_returns <= np.percentile(asset_returns, 5)].mean():.2%}']
]
table = ax6.table(cellText=table_data, cellLoc='left', loc='center', 
                 colWidths=[0.6, 0.4])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 1.5)
ax6.text(0.5, 0.9, 'Key Risk Metrics', transform=ax6.transAxes, 
        ha='center', fontsize=12, fontweight='bold')

plt.suptitle(f'Risk Analysis Dashboard - {ticker}', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Summary and Insights

This notebook demonstrated comprehensive risk analysis techniques:

1. **Volatility Analysis**: Historical and rolling volatility provide insights into changing risk levels

2. **Value at Risk (VaR)**: Multiple calculation methods show potential losses at different confidence levels

3. **Conditional VaR**: Provides expected loss beyond VaR threshold

4. **Beta Analysis**: Measures systematic risk relative to market

5. **Drawdown Analysis**: Shows historical peak-to-trough losses and recovery periods

6. **Risk-Adjusted Metrics**: Sharpe, Sortino, and Calmar ratios evaluate return per unit of risk

### Key Takeaways:
- Risk is multi-dimensional and requires various metrics for complete understanding
- Historical analysis provides valuable insights but has limitations
- Different risk metrics serve different purposes in portfolio management
- Visual analysis helps identify patterns and outliers in risk behavior